The same data preparation as before

In [1]:
import numpy as np
import pandas as pd

# 读数据函数
# testee为被测试者代号 有arc ctj lky
# dType为数据类型 有Mag和Phase

def ReadData(testee, dType, fileNum):
    dataList = list()
    for i in range(0, fileNum):
        fileName = 'D' + str(i)
        pathName = 'data/' + testee + '/' + dType + '/' + fileName + '.csv'
        data = pd.read_csv(pathName, header = None)
        dataList.append(data)
    #print(len(dataList))
    #print(dataList[50])
    return dataList

arcMag = ReadData('arc', 'Mag', 351)
ctjMag = ReadData('ctj', 'Mag', 162)
lkyMag = ReadData('lky', 'Mag', 350)

In [2]:
# 先删除不足500列的数据
# 再把剩余数据中多余的部分截去

dataMag = list()

def dataMagPre(dataName):
    l = len(dataName)
    for i in range(0, l):
        rowL = dataName[i].shape[1]
        if rowL < 500:
            continue
        else:
            temp = np.array(dataName[i])[:,:500]
            dataMag.append(temp)
            
dataMagPre(arcMag)
print('Number of class arc : ', len(dataMag))
dataMagPre(ctjMag)
print('Number of class arc + ctj : ', len(dataMag))
dataMagPre(lkyMag)
print('Number of class arc + ctj + lky : ', len(dataMag))

Number of class arc :  341
Number of class arc + ctj :  503
Number of class arc + ctj + lky :  831


In [3]:
dataMagArray = np.array(dataMag)

for i in range(0, 831):
    for j in range(0, 57):
        for k in range(0, 500):
            if dataMagArray[i, j, k] > 356:
                dataMagArray[i, j, k] = 356

mi = ma = dataMagArray[0, 0, 0]
print(ma)

for i in range(0, 831):
    for j in range(0, 57):
        for k in range(0, 500):
            if dataMagArray[i, j, k] < mi:
                mi = dataMagArray[i, j, k]
            if dataMagArray[i, j, k] > ma:
                ma = dataMagArray[i, j, k]

print('min:', mi, '; max:', ma)

115.0043
min: 0.0 ; max: 356.0


In [4]:
NdataMagArray=np.zeros((1662,57,250))

xxx=0

for i in range(0,1662):
    xxx=int(i/2)
    for j in range(0,57):
        if i%2==0:
            for k in range(0,250):
                NdataMagArray[i,j,k]=dataMagArray[xxx,j,k]
        if i%2==1:
            for k in range(250,500):
                NdataMagArray[i,j,k-250]=dataMagArray[xxx,j,k]
                
dataMagArray = NdataMagArray

dataMagArray = dataMagArray.astype('float32') * (256 / 356)

In [5]:
print('Data shape:', dataMagArray.shape)
pd.DataFrame(dataMagArray[0,:10,:13])

Data shape: (1662, 57, 250)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,82.699722,88.764549,83.070992,102.448540,142.020172,80.696449,78.629036,123.760612,75.509071,89.475380,106.115524,92.565933,85.681763
1,82.784142,87.067688,85.003075,105.443382,143.764496,84.917862,71.194679,119.188705,75.939232,89.749504,108.555717,85.100365,86.987434
2,85.100365,81.487175,88.985672,106.587181,143.604355,81.287048,71.784126,120.543274,77.249008,92.942238,111.077248,90.189156,88.741249
3,84.184013,78.786728,84.137924,104.339058,134.448822,79.349144,75.628799,115.855675,78.246689,90.601067,105.050323,89.145309,86.441849
4,85.585114,86.513550,89.515793,105.489983,134.909561,76.731827,77.529671,113.043060,78.816216,95.187920,110.454002,88.191856,87.909897
5,89.307610,90.051453,90.891655,105.808113,140.629791,79.199135,80.211205,117.662636,85.018318,98.127678,112.552559,93.985153,92.056160
6,107.550705,105.835007,110.182190,125.746124,155.327499,88.715073,95.931969,134.040527,98.359230,118.022476,130.284363,108.593765,108.743690
7,119.639076,118.828079,120.611954,137.513855,167.105576,101.089363,106.061951,147.069778,108.593765,131.266953,142.347504,119.762192,121.791641
8,124.958206,122.585091,125.894035,141.114319,164.071701,96.598076,111.367836,143.279892,114.122070,137.052475,147.361359,125.051254,125.125679
9,124.213142,119.123627,125.373413,137.240952,148.283112,87.918747,110.245468,138.553467,112.129082,134.258347,142.695847,122.698997,124.885719


Save the array as RBG images

In [6]:
import PIL.Image as Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.io import loadmat

In [7]:
# 图片数据分组情况
# 训练集、验证集、测试集占比分别为70% 15% 15%

# 1662 = 628 + 324 + 656
#   （478+102+102）
#          （226+49+49）
#                （460+98+98）

# class train       valid       test
# arc   [0,478]     [478,580]   [580,682]
# ctj   [682,908]   [908,957]   [957,1006]
# lky   [1006,1466] [1466,1564] [1564,1662]

In [9]:
imgsList = list()

def matrixToImage(num1, num2, cla, typ):
    for i in range(num1, num2):
        tmpImg = Image.fromarray(dataMagArray[i])
        tmpImg = tmpImg.convert('RGB')
        imgsList.append(tmpImg)
        filepath = 'imgs/' + cla + '/' + typ + '/' + str(i) + '.jpg'
        tmpImg.save(filepath)

In [10]:
'''
i = 0
cla = 'train'
typ = 'arc'

tmpImg = Image.fromarray(dataMagArray[i])
tmpImg = tmpImg.convert('RGB')
filepath = 'imgs/' + cla + '/' + typ + '/' + str(i) + '.jpg'
tmpImg.save(filepath)
'''

"\ni = 0\ncla = 'train'\ntyp = 'arc'\n\ntmpImg = Image.fromarray(dataMagArray[i])\ntmpImg = tmpImg.convert('RGB')\nfilepath = 'imgs/' + cla + '/' + typ + '/' + str(i) + '.jpg'\ntmpImg.save(filepath)\n"

In [11]:
matrixToImage(0, 478, 'train', 'arc')
matrixToImage(478, 580, 'valid', 'arc')
matrixToImage(580, 682, 'test', 'arc')

matrixToImage(682, 908, 'train', 'ctj')
matrixToImage(908, 957, 'valid', 'ctj')
matrixToImage(957, 1006, 'test', 'ctj')

matrixToImage(1006, 1466, 'train', 'lky')
matrixToImage(1466, 1564, 'valid', 'lky')
matrixToImage(1564, 1662, 'test', 'lky')